# Transformery

### Transformery

0. Twórcy: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Łukasz Kaiser, Illia Polosukhin,
Publikacja: "Attention Is All You Need", Advances in Neural Information Processing Systems (NeurIPS, 2017),
arXiv:1706.03762

1. Definicja. Transformery to architektura sieci neuronowych zaprojektowana do przetwarzania sekwencji danych.

2. Mechanizm uwagi ("attention"). Transformery opierają się na mechanizmie "self-attention", który pozwala modelowi skupiać się na różnych częściach sekwencji wejściowej, niezależnie od ich odległości.

3. Brak rekurencji: W przeciwieństwie do RNN i LSTM, transformery nie używają rekurencji, co przyspiesza obliczenia i ułatwia równoległe przetwarzanie.

4. Skalowalność. Transformery są wysoce skalowalne i mogą być łatwo dostosowywane do dużych zbiorów danych, co czyni je idealnymi do zadań takich jak tłumaczenie maszynowe czy generowanie tekstu.

5. Encoder-Decoder. Standardowy transformer składa się z dwóch głównych części:

- Encoder: Przetwarza sekwencję wejściową.

- Decoder: Generuje sekwencję wyjściową na podstawie zakodowanej reprezentacji.

6. "Positional Encoding". Ponieważ transformery nie mają wbudowanej informacji o kolejności elementów w sekwencji, dodaje się kodowanie pozycyjne, aby zachować informację o pozycji.

7. Zastosowania. Transformery są używane w zadaniach takich jak tłumaczenie maszynowe, generowanie tekstu (np. GPT), klasyfikacja tekstu (np. BERT) i analiza obrazów (np. Vision Transformers).

8. Efektywność. Dzięki równoległemu przetwarzaniu sekwencji Transformery są szybsze w trenowaniu niż RNN/LSTM, zwłaszcza na sprzęcie GPU/TPU.

9. "Pre-training" i "fine-tuning". Transformery są często wstępnie trenowane na dużych zbiorach danych, a następnie dostrajane do konkretnych zadań (np. BERT, GPT).

10. Dominacja. Transformery zrewolucjonizowały przetwarzanie języka naturalnego (NLP) i stały się podstawą wielu nowoczesnych modeli, takich jak GPT, BERT czy T5.

In [2]:
# Przykład

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, GlobalMaxPooling1D
from tensorflow.keras.layers import Embedding, MultiHeadAttention, LayerNormalization, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Parametry
vocab_size = 20000  # Liczba słów w słowniku
maxlen = 200        # Maksymalna długość sekwencji
embed_dim = 32      # Wymiar embeddingu
num_heads = 2       # Liczba głów w mechanizmie uwagi
ff_dim = 32         # Wymiar warstwy feed-forward
batch_size = 32
epochs = 2

# Wczytanie danych IMDB
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocab_size)

# Przygotowanie danych (padding)
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)

# Definicja warstwy Transformer Encoder
class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential([
            Dense(ff_dim, activation="relu"),
            Dense(embed_dim),
        ])
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, inputs, training = False):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

# Budowa modelu
inputs = Input(shape=(maxlen,))
embedding_layer = Embedding(vocab_size, embed_dim)(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)(embedding_layer)
pooling_layer = GlobalMaxPooling1D()(transformer_block)
outputs = Dense(1, activation="sigmoid")(pooling_layer)

model = Model(inputs, outputs)
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

# Trenowanie modelu
history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_test, y_test))

# Ocena modelu
loss, accuracy = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {accuracy:.4f}")

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 200)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_1 (Embedding)              │ (None, 200, 32)             │         640,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ transformer_block_1                  │ (None, 200, 32)             │          10,656 │
│ (TransformerBlock)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_max_pooling1d                 │ (None, 32)                  │               0 │
│ (GlobalMaxPooling1D)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 650,689 (2.48 MB)

 Trainable params: 650,689 (2.48 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/2
782/782 ━━━━━━━━━━━━━━━━━━━━ 112s 137ms/step - accuracy: 0.7533 - loss: 0.4904 - val_accuracy: 0.8616 - val_loss: 0.3168
Epoch 2/2
782/782 ━━━━━━━━━━━━━━━━━━━━ 159s 160ms/step - accuracy: 0.9137 - loss: 0.2153 - val_accuracy: 0.8617 - val_loss: 0.3306
782/782 ━━━━━━━━━━━━━━━━━━━━ 29s 37ms/step - accuracy: 0.8625 - loss: 0.3321
Test Accuracy: 0.8617


### Uwagi

1. Dane. Używamy zbioru danych IMDB, który zawiera 50 000 recenzji filmowych oznaczonych jako pozytywne lub negatywne.

2. Przygotowanie danych. Sekwencje tekstowe są przekształcane na sekwencje liczb całkowitych i dopełniane do stałej długości (maxlen).

3. TransformerBlock: Definiujemy warstę Transformer Encoder (poprzez określenie odpowiedniej klasy), która składa się z mechanizmu "Multi-Head Attention", warstwy "feed-forward" i normalizacji.

4. Model. Model składa się z warstwy osadzenia ("embeddingu"), warstwy Transformer, globalnego "max pooling" i warstwy wyjściowej.

5. Trening. Model jest trenowany przez dwie epoki z użyciem optymalizatora Adam i funkcji straty binary cross-entropy.

6. Wyniki. Model osiąga dokładność (accuracy) na zbiorze testowym, która jest wypisywana na końcu.

### Dlaczego Transformery są lepsze niż LSTM/RNN:

1. Równoległe przetwarzanie. Transformery przetwarzają całe sekwencje równolegle, co znacznie przyspiesza obliczenia.

2. Długie zależności. Mechanizm self-attention pozwala Transformatorom efektywnie modelować długoterminowe zależności w danych.

3. Skalowalność. Transformery są łatwe do skalowania na dużych zbiorach danych i potężnym sprzęcie (GPU/TPU).

4. Uniwersalność. Transformery są stosowane nie tylko w NLP, ale także w przetwarzaniu obrazów, dźwięku i innych dziedzinach.